In [1]:
import os
from math import ceil
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ROOT = "Oral Cancer Prediction"
DATA_PATH = os.path.join(ROOT, "assets", "dataset")

IMG_SIZE = (224, 224)

BATCH_SIZE = 32

In [ ]:
ds = image_dataset_from_directory(
    DATA_PATH, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=123
)

Found 719 files belonging to 2 classes.


In [ ]:
CLASS_NAMES = ds.class_names
print(CLASS_NAMES)

class_counts = {
    0: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[0])).__len__(),
    1: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[1])).__len__(),
}
print(class_counts)

['cancer', 'normal']
{0: 188, 1: 531}


In [5]:
for img_batch, label_batch in ds.take(1):
    print(img_batch.shape)
    print(img_batch[0].shape)

(32, 224, 224, 3)
(224, 224, 3)


Batches of 32 images of size (224, 224) in 3 color channels.


## Visualise some of the images from our dataset


In [6]:
# plt.figure(figsize=(6,6))
# for img_batch, label_batch in ds.take(1):
#     for i in range(12):
#         plt.subplot(3, 4, i+1)
#         plt.imshow(img_batch[i].numpy().astype('uint8'))
#         plt.title(CLASS_NAMES[label_batch[i]])
#         plt.axis('off')

## Split dataset


In [ ]:
train_split = 0.8
val_split = 0.1
train_size = int(len(ds) * train_split)
val_size = ceil(len(ds) * val_split)
print(f"Number of batches:   {len(ds)}")
print(f"Training batches:    {train_size}")
print(f"Validating batches:  {val_size}")
print(f"Testing batches:     {len(ds) - train_size - val_size}")

Number of batches:   23
Training batches:    18
Validating batches:  3
Testing batches:     2


In [8]:
train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)
val_ds = test_ds.take(val_size)
test_ds = test_ds.skip(val_size)

Creating a function to do all these things with shuffling


In [ ]:
def dataset_partitions(
    ds, train_split=0.8, val_split=0.1, shuffle=True, shuffle_size=1000
):
    if shuffle:
        ds.shuffle(shuffle_size)

    ds_size = int(ds.reduce(0, lambda x, _: x + 1).numpy())
    train_size = int(ds_size * train_split)
    val_size = ceil(ds_size * val_split)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [10]:
train_ds, val_ds, test_ds = dataset_partitions(ds)

print(f"Number of batches  : {len(ds)}")
print(f"Training batches   : {len(train_ds)}")
print(f"Validating batches : {len(val_ds)}")
print(f"Testing batches    : {len(test_ds)}")

Number of batches  : 23
Training batches   : 18
Validating batches : 3
Testing batches    : 2


In [11]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Bulding a model


In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

## Manual CNN model


In [ ]:
input_shape = (BATCH_SIZE, *IMG_SIZE, 3)
n_classes = 2

model = models.Sequential(
    [
        layers.Resizing(*IMG_SIZE),
        layers.Rescaling(1.0 / 255),
        data_augmentation,
        layers.Conv2D(
            32, kernel_size=(3, 3), activation="relu", input_shape=input_shape
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.build(input_shape=input_shape)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (32, 222, 222, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (32, 111, 111, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 109, 109, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (32, 54, 54, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 52, 52, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (32, 26, 26, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 24, 24, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (32, 12, 12, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (32, 10, 10, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (32, 5, 5, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (32, 3, 3, 64)         │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (32, 1, 1, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,329 (669.25 KB)

 Trainable params: 171,329 (669.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model.fit(train_ds, validation_data=val_ds, batch_size=BATCH_SIZE, epochs=10)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 871ms/step - accuracy: 0.6510 - auc: 0.4756 - loss: 0.6625 - val_accuracy: 0.7500 - val_auc: 0.3157 - val_loss: 0.5927
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 50ms/step - accuracy: 0.7328 - auc: 0.4492 - loss: 0.6019 - val_accuracy: 0.7500 - val_auc: 0.3197 - val_loss: 0.5706
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.7328 - auc: 0.4557 - loss: 0.5939 - val_accuracy: 0.7500 - val_auc: 0.2969 - val_loss: 0.5733
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.7328 - auc: 0.4814 - loss: 0.5926 - val_accuracy: 0.7500 - val_auc: 0.3142 - val_loss: 0.5706
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.7328 - auc: 0.4549 - loss: 0.5943 - val_accuracy: 0.7500 - val_auc: 0.3302 - val_loss: 0.5708
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.7328 - auc: 0.4556 - loss: 0.5927 - val_accuracy: 0.7500 - val_auc: 0.3406 - val_loss: 0.5692
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47

In [16]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step 
              precision    recall  f1-score   support

      cancer     0.2500    1.0000    0.4000        24
      normal     0.0000    0.0000    0.0000        72

    accuracy                         0.2500        96
   macro avg     0.1250    0.5000    0.2000        96
weighted avg     0.0625    0.2500    0.1000        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Transfer Learning using ResNet50


In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,637,313 (93.98 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [18]:
# tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Compile
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model.fit(train_ds, validation_data=val_ds, batch_size=BATCH_SIZE, epochs=10)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 324ms/step - accuracy: 0.6488 - auc_1: 0.5178 - loss: 0.7117 - val_accuracy: 0.7500 - val_auc_1: 0.6212 - val_loss: 0.6369
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.7262 - auc_1: 0.4744 - loss: 0.6218 - val_accuracy: 0.7500 - val_auc_1: 0.6942 - val_loss: 0.5627
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - accuracy: 0.7285 - auc_1: 0.4729 - loss: 0.6062 - val_accuracy: 0.7500 - val_auc_1: 0.7109 - val_loss: 0.5469
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 151ms/step - accuracy: 0.7284 - auc_1: 0.5222 - loss: 0.5876 - val_accuracy: 0.7500 - val_auc_1: 0.7037 - val_loss: 0.5413
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.7299 - auc_1: 0.5129 - loss: 0.5912 - val_accuracy: 0.7500 - val_auc_1: 0.7141 - val_loss: 0.5372
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.7275 - auc_1: 0.5582 - loss: 0.5792 - val_accuracy: 0.7500 - val_auc_1: 0.7135 - val_loss: 0.5347
Epoch 7/10
18/1

In [20]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step
              precision    recall  f1-score   support

      cancer     0.2500    1.0000    0.4000        24
      normal     0.0000    0.0000    0.0000        72

    accuracy                         0.2500        96
   macro avg     0.1250    0.5000    0.2000        96
weighted avg     0.0625    0.2500    0.1000        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Dealing with Imbalanced datset

Methods:-

1. using custom weights
2. undersampling larger dataset
3. duplicate oversampling smaller dataset
4. SMOTE oversampling smaller dataset (mod = imblearn)
5. ensemble technique
6. focal loss


### M2. Undersampling


In [ ]:
min_class_count = min(class_counts.values())

under_ds = (
    ds.unbatch()
    .filter(lambda x, y: y == 0)
    .take(min_class_count)
    .concatenate(ds.unbatch().filter(lambda x, y: y == 1).take(min_class_count))
    .batch(BATCH_SIZE)
    .shuffle(500)
)

In [22]:
from collections import Counter

counter = Counter()
for _, labels in under_ds.unbatch():
    counter[int(labels.numpy())] += 1

print("Balanced class counts:", counter)

Balanced class counts: Counter({1: 188, 0: 188})


In [ ]:
print("No. of batches:", end=" ")
print(int(under_ds.reduce(0, lambda x, _: x + 1).numpy()))

No. of batches: 12


In [24]:
train_under_ds, val_under_ds, test_under_ds = dataset_partitions(under_ds)

train_under_ds = train_under_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_under_ds = val_under_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_under_ds = test_under_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
input_shape = (BATCH_SIZE, *IMG_SIZE, 3)

model_under = models.Sequential(
    [
        layers.Resizing(*IMG_SIZE),
        layers.Rescaling(1.0 / 255),
        data_augmentation,
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model_under.build(input_shape=input_shape)
model_under.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_2 (Resizing)           │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (32, 222, 222, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (32, 111, 111, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (32, 109, 109, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (32, 54, 54, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (32, 52, 52, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (32, 26, 26, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (32, 24, 24, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (32, 12, 12, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (32, 10, 10, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (32, 5, 5, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (32, 3, 3, 64)         │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (32, 1, 1, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (32, 64)               │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,329 (669.25 KB)

 Trainable params: 171,329 (669.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile
model_under.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model_under.fit(
    train_under_ds,
    validation_data=val_under_ds,
    batch_size=BATCH_SIZE,
    steps_per_epoch=4,
    epochs=2,
)

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1732 - auc_3: 0.0725 - loss: 0.7068    

KeyboardInterrupt: 

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_under_ds], axis=0)
y_pred_probs = model_under.predict(val_under_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

### M3. Oversampling


In [ ]:
max_class_count = max(class_counts.values())
count_value = {val: key for key, val in class_counts.items()}

over_ds = (
    ds.unbatch()
    .filter(lambda x, y: y != count_value[max_class_count])
    .repeat()
    .take(max_class_count)
    .concatenate(
        ds.unbatch()
        .filter(lambda x, y: y == count_value[max_class_count])
        .take(max_class_count)
    )
    .batch(BATCH_SIZE)
    .shuffle(2 * max_class_count)
)

In [ ]:
from collections import Counter

counter = Counter()
for _, labels in over_ds.unbatch():
    counter[int(labels.numpy())] += 1
print("Balanced class counts:", counter)

print("No. of batches:", end=" ")
print(int(over_ds.reduce(0, lambda x, _: x + 1).numpy()))

Balanced class counts: Counter({1: 531, 0: 531})
No. of batches: 34


In [31]:
train_over_ds, val_over_ds, test_over_ds = dataset_partitions(over_ds)

train_over_ds = train_over_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_over_ds = val_over_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_over_ds = test_over_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_over = models.Model(inputs, outputs)

model_over.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_4 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,637,313 (93.98 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Compile
model_over.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model_over.fit(
    train_over_ds,
    validation_data=val_over_ds,
    batch_size=BATCH_SIZE,
    steps_per_epoch=5,
    epochs=5,
)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.3417 - auc_5: 0.1786 - loss: 1.9990

KeyboardInterrupt: 

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_under_ds], axis=0)
y_pred_probs = model_under.predict(val_under_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))